# Sinopia Entity Resource Template Classification

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

import datetime
import os
import rdflib
import numpy as np
import pandas as pd
import tensorflow as tf

## Setup
Create two graphs, one for testing and one for training

In [14]:
LDP = rdflib.Namespace('http://www.w3.org/ns/ldp#')
SINOPIA_TRAIN = rdflib.ConjunctiveGraph()
SINOPIA_TRAIN.namespace_manager.bind("ldp", LDP)
SINOPIA_TEST = rdflib.ConjunctiveGraph()
SINOPIA_TEST.namespace_manager.bind("ldp", LDP)
SINOPIA_TESTING_PATH = "/Users/jpnelson/2019/sinopia-data/2019/08/28/test/"
for filename in next(os.walk(SINOPIA_TESTING_PATH))[2]:
    SINOPIA_TEST.parse(os.path.join(SINOPIA_TESTING_PATH, filename), format='turtle')
SINOPIA_TRAIN_PATH = "/Users/jpnelson/2019/sinopia-data/2019/08/28/train/"
for filename in next(os.walk(SINOPIA_TRAIN_PATH))[2]:
    SINOPIA_TRAIN.parse(os.path.join(SINOPIA_TRAIN_PATH, filename), format='turtle')


In [17]:
print(f"Testing triples: {len(SINOPIA_TEST):,}, Training triples: {len(SINOPIA_TRAIN):,}")

Testing triples: 446, Training triples: 2,602
